In [40]:
import pandas as pd
from .\Users\vonke\Documents\Ironhack\Ironhack Week 2\Day 1\Shark-Attacks\functions.py import my_function

SyntaxError: unexpected character after line continuation character (4110935569.py, line 2)

In [ ]:
my_function()